In [15]:
# ===================== CELL 0 — Bootstrap =====================
import os
import sys
from pathlib import Path

# Find repo root (folder containing "src/imgofup")
p = Path.cwd().resolve()
REPO_ROOT = None
for candidate in [p, *p.parents]:
    if (candidate / "src" / "imgofup").is_dir():
        REPO_ROOT = candidate
        break
if REPO_ROOT is None:
    raise RuntimeError("Could not find repo root (no 'src/imgofup' found).")

SRC_DIR = REPO_ROOT / "src"
if str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

# Tell config where the repo root is (paths.py uses it)
os.environ["PROJ_ROOT"] = str(REPO_ROOT)

print("Repo root:", REPO_ROOT)
print("sys.path[0]:", sys.path[0])
print("PROJ_ROOT:", os.environ["PROJ_ROOT"])


Repo root: /Users/amirdonyadide/Documents/GitHub/IMGOFUP
sys.path[0]: /opt/anaconda3/envs/thesis/lib/python311.zip
PROJ_ROOT: /Users/amirdonyadide/Documents/GitHub/IMGOFUP


In [16]:
# ===================== CELL 1 — Config + I/O paths (Excel raw input) =====================
from pathlib import Path
import os

from imgofup.config import paths as CONFIG

DATA_DIR = Path(CONFIG.PATHS.DATA_DIR) / "userstudy"
DATA_DIR.mkdir(parents=True, exist_ok=True)

# Raw input (Excel)
RAW_XLSX = Path(CONFIG.PATHS.USER_STUDY_XLSX).expanduser().resolve()
RAW_SHEET = getattr(CONFIG.PATHS, "RESPONSES_SHEET", "Responses")

# Final outputs (cleaned dataset)
OUT_XLSX = DATA_DIR / "UserStudy.cleaned.xlsx"
OUT_CSV  = DATA_DIR / "UserStudy.cleaned.csv"

print("DATA_DIR :", DATA_DIR)
print("RAW_XLSX :", RAW_XLSX, "| exists:", RAW_XLSX.is_file())
print("SHEET   :", RAW_SHEET)
print("OUT_XLSX:", OUT_XLSX)
print("OUT_CSV :", OUT_CSV)

if not RAW_XLSX.is_file():
    raise FileNotFoundError(
        f"User study Excel not found:\n  {RAW_XLSX}\n"
        "Fix: set PROJ_ROOT correctly in 00_setup.ipynb OR update PATHS.USER_STUDY_XLSX."
    )

print("OPENAI_API_KEY set:", bool(os.getenv("OPENAI_API_KEY")))


DATA_DIR : /Users/amirdonyadide/Documents/GitHub/IMGOFUP/data/userstudy
RAW_XLSX : /Users/amirdonyadide/Documents/GitHub/IMGOFUP/data/userstudy/UserStudy.xlsx | exists: True
SHEET   : Responses
OUT_XLSX: /Users/amirdonyadide/Documents/GitHub/IMGOFUP/data/userstudy/UserStudy.cleaned.xlsx
OUT_CSV : /Users/amirdonyadide/Documents/GitHub/IMGOFUP/data/userstudy/UserStudy.cleaned.csv
OPENAI_API_KEY set: True


In [17]:
# ===================== CELL 2 — Ensure runtime deps (optional) =====================
import sys, subprocess, importlib

def ensure_import(pkg_import: str, pip_name: str | None = None):
    pip_name = pip_name or pkg_import
    try:
        importlib.import_module(pkg_import)
        print(f"{pkg_import} available")
    except Exception:
        print(f"Installing {pip_name} ...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", pip_name])
        importlib.import_module(pkg_import)
        print(f"{pkg_import} installed")

# Required for LLM + env
ensure_import("openai", "openai")
ensure_import("dotenv", "python-dotenv")

# Optional quality-of-life
ensure_import("tqdm", "tqdm")
ensure_import("langdetect", "langdetect")


openai available
dotenv available
tqdm available
langdetect available


In [18]:
# ===================== CELL 3 — Load .env + init OpenAI client =====================
import os
from dotenv import load_dotenv

from imgofup.userstudy.prompt_cleaning import get_openai_client, LLMConfig

# Load .env at repo root (PROJ_ROOT points there)
load_dotenv(dotenv_path=Path(os.environ["PROJ_ROOT"]) / ".env")

# Fail fast if missing
if not os.getenv("OPENAI_API_KEY"):
    raise RuntimeError(
        "OPENAI_API_KEY not found.\n"
        "Fix: create .env in repo root and add:\n"
        "OPENAI_API_KEY=sk-...\n"
    )

client = get_openai_client()

# Central LLM config for this notebook
LLM = LLMConfig(
    model="gpt-4o-mini",
    temperature=0.0,
    max_retries=3,
    retry_sleep_s=0.8,
)

print("OpenAI client ready")
print("LLM config:", LLM)


OpenAI client ready
LLM config: LLMConfig(model='gpt-4o-mini', temperature=0.0, max_retries=3, retry_sleep_s=0.8)


In [ ]:
# ===================== CELL 4 — Load raw Excel + basic normalization =====================
import pandas as pd

from imgofup.userstudy.prompt_cleaning import normalize_empty_to_nan

# Load raw Excel (sheet name comes from CELL 1)
df_raw = pd.read_excel(RAW_XLSX, sheet_name=RAW_SHEET)

# Column that contains the raw user text
RAW_TEXT_COL = "free_text"   # change if your Excel uses another name

if RAW_TEXT_COL not in df_raw.columns:
    raise KeyError(
        f"Expected column '{RAW_TEXT_COL}' in sheet '{RAW_SHEET}'.\n"
        f"Available columns: {list(df_raw.columns)}"
    )

df = df_raw.copy()

# Normalize empty / junk strings → NaN
df[RAW_TEXT_COL] = df[RAW_TEXT_COL].apply(normalize_empty_to_nan)

print("Rows:", len(df))
print("Non-empty free_text:", int(df[RAW_TEXT_COL].notna().sum()))

df.head(5)


Rows: 786
Non-empty free_text: 786


,timestamp,assignment_id,participant_id,tile_id,prompt_id,complete,remove,free_text,cleaned_text,param_value,operator,intensity,threshold_exist,threshold_known,conflict
0,2025-11-27 21:20:30.892,b9b3eea7-3fde-4320-8eb1-2f397fdb787d,0fa332ca-ab78-4e18-8d9c-74a21bdacc81,1304,1,True,False,الگویی رو نمیبینم اجتماع داریم ولی عمومی نیست ...,Union few of the buildings.,0.000,aggregate,medium,False,NaN,False
1,2025-11-27 20:27:27.337,cb0ed21c-6777-48c3-b7be-425069f4f585,17b7bd76-8792-4806-b2a7-88e70cfb822d,433,2,True,True,remove most of the smaller and closely spaced ...,Remove most of the smaller and closely spaced ...,0.094,select,high,True,True,False
2,2025-12-06 20:11:54.205,87db583a-fca6-4858-b4bb-f25d05530b86,fc62e5f9-9578-4aeb-b194-2e52c2047769,1457,3,True,False,Remove small buildings and eliminate narrow an...,Remove small buildings and eliminate narrow an...,17.805,select,low,True,True,False
3,2025-11-27 19:57:43.721,c82c8e11-48d8-4f74-af3f-d4aa95744c38,bc57129a-ef70-43b7-a511-4654c950d26c,1617,4,True,True,I want to use the map on the left side because...,NaN,0.000,aggregate,low,False,NaN,True
4,2025-11-27 19:59:48.805,2e6ba9a9-3beb-4bfa-979c-7d51eb097214,bc57129a-ef70-43b7-a511-4654c950d26c,1532,5,True,True,I want to use the map on the left side because...,NaN,6.309,displace,medium,False,NaN,True


In [ ]:
# ===================== CELL 5 — Step 1: Prompt detection =====================
import pandas as pd

from imgofup.userstudy.prompt_cleaning import looks_like_prompt_llm, apply_series, normalize_empty_to_nan

RUN_PROMPT_DETECT = True
USE_TQDM = True  # set False if you don't want progress bars

# ensure normalized
df[RAW_TEXT_COL] = df[RAW_TEXT_COL].apply(normalize_empty_to_nan)

if RUN_PROMPT_DETECT:
    df["step1_is_prompt"] = apply_series(
        df[RAW_TEXT_COL],
        lambda x: looks_like_prompt_llm(x, client=client, cfg=LLM),
        use_tqdm=USE_TQDM,
    ).astype("boolean")
else:
    df["step1_is_prompt"] = df[RAW_TEXT_COL].notna().astype("boolean")

df["step1_text"] = df[RAW_TEXT_COL].where(df["step1_is_prompt"], pd.NA).astype("string")

df["drop_reason"] = pd.NA
df.loc[df[RAW_TEXT_COL].isna(), "drop_reason"] = "empty_free_text"
df.loc[df[RAW_TEXT_COL].notna() & (~df["step1_is_prompt"]), "drop_reason"] = "not_prompt_like"

print("Prompt-like rows:", int(df["step1_is_prompt"].sum()))
print("Dropped rows    :", int((~df["step1_is_prompt"]).sum()))
df[[RAW_TEXT_COL, "step1_is_prompt", "drop_reason", "step1_text"]].head(10)


Applying:   0%|          | 0/786 [00:00<?, ?it/s]

Prompt-like rows: 624
Dropped rows    : 162


,free_text,step1_is_prompt,drop_reason,step1_text
0,الگویی رو نمیبینم اجتماع داریم ولی عمومی نیست ...,False,not_prompt_like,<NA>
1,remove most of the smaller and closely spaced ...,True,<NA>,remove most of the smaller and closely spaced ...
2,Remove small buildings and eliminate narrow an...,True,<NA>,Remove small buildings and eliminate narrow an...
3,I want to use the map on the left side because...,False,not_prompt_like,<NA>
4,I want to use the map on the left side because...,False,not_prompt_like,<NA>
5,Bundle nearby buildings into larger blocks and...,True,<NA>,Bundle nearby buildings into larger blocks and...
6,Some small details below a certain threshold h...,False,not_prompt_like,<NA>
7,اینجا بر حسب مساحت پلیگون ها پاک شدن شدن,False,not_prompt_like,<NA>
8,I want to use the map on the left side because...,False,not_prompt_like,<NA>
9,elimination of repeated blocks,False,not_prompt_like,<NA>


In [7]:
# ===================== CELL 6 — Step 2: Detect language + translate to English if needed =====================
import numpy as np

from imgofup.userstudy.prompt_cleaning import (
    safe_detect_lang,
    translate_to_english,
    apply_series,
)

# --- Language detection ---
df["step1_lang"] = df["step1_text"].apply(
    lambda x: safe_detect_lang(x) if isinstance(x, str) else "unknown"
)

# --- Decide which rows need translation ---
need_translate = (
    df["step1_is_prompt"]
    & df["step1_text"].notna()
    & (df["step1_lang"] != "en")
)

# --- Initialize output column ---
df["step2_text"] = np.nan

# English prompts → pass through
mask_en = df["step1_is_prompt"] & (df["step1_lang"] == "en")
df.loc[mask_en, "step2_text"] = df.loc[mask_en, "step1_text"]

# Non-English prompts → translate
if need_translate.any():
    df.loc[need_translate, "step2_text"] = apply_series(
        df.loc[need_translate, "step1_text"],
        lambda x: translate_to_english(x, client=client, cfg=LLM),
        use_tqdm=True,
        desc="Translating to English",
    )

print("Need translate:", int(need_translate.sum()))
df[["step1_lang", "step1_text", "step2_text"]].head(10)


/var/folders/6r/lw1cp6x117zfdkxgl862fymc0000gn/T/ipykernel_81669/2607224222.py:27: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '<StringArray>
[           'remove most of the smaller and closely spaced buildings, keeping only larger, more isolated ones with slightly simplified shapes',
             'Remove small buildings and eliminate narrow and tiny shapes. Keep the larger buildings to create a cleaner, less cluttered map.',
                                                                'Bundle nearby buildings into larger blocks and simplify the resulting shapes',
                                                                   'Make a space of 5sqm between the polygons in the coordinates of (180, 88)',
                                                                                                                 'Eliminate the old buildings',
                                                      

Translating to English:   0%|          | 0/14 [00:00<?, ?it/s]

Need translate: 14


,step1_lang,step1_text,step2_text
0,unknown,<NA>,NaN
1,en,remove most of the smaller and closely spaced ...,remove most of the smaller and closely spaced ...
2,en,Remove small buildings and eliminate narrow an...,Remove small buildings and eliminate narrow an...
3,unknown,<NA>,NaN
4,unknown,<NA>,NaN
5,en,Bundle nearby buildings into larger blocks and...,Bundle nearby buildings into larger blocks and...
6,unknown,<NA>,NaN
7,unknown,<NA>,NaN
8,unknown,<NA>,NaN
9,unknown,<NA>,NaN


In [8]:
# ===================== CELL 7 — Define cleaned_text =====================
import numpy as np

# We skip the grammar LLM step. Treat translated text as final cleaned text.
df["cleaned_text"] = df["step2_text"].where(df["step2_text"].notna(), np.nan)

print("Cleaned prompts:", int(df["cleaned_text"].notna().sum()))
df[["step2_text", "cleaned_text"]].head(10)


Cleaned prompts: 624


,step2_text,cleaned_text
0,NaN,NaN
1,remove most of the smaller and closely spaced ...,remove most of the smaller and closely spaced ...
2,Remove small buildings and eliminate narrow an...,Remove small buildings and eliminate narrow an...
3,NaN,NaN
4,NaN,NaN
5,Bundle nearby buildings into larger blocks and...,Bundle nearby buildings into larger blocks and...
6,NaN,NaN
7,NaN,NaN
8,NaN,NaN
9,NaN,NaN


In [19]:
# ===================== CELL 8 — Threshold columns (placeholders; no LLM) =====================
# If you don't want threshold detection, set safe defaults.
# This prevents later cells from failing if they expect these columns.

if "threshold_exist" not in df.columns:
    df["threshold_exist"] = False
if "threshold_evidence" not in df.columns:
    df["threshold_evidence"] = ""

if "threshold_known" not in df.columns:
    df["threshold_known"] = False
if "threshold_known_evidence" not in df.columns:
    df["threshold_known_evidence"] = ""

print("Threshold columns created as placeholders (all False).")
df[["cleaned_text", "threshold_exist", "threshold_known"]].head(10)


Threshold columns created as placeholders (all False).


,cleaned_text,threshold_exist,threshold_known
0,NaN,False,NaN
1,remove most of the smaller and closely spaced ...,True,True
2,Remove small buildings and eliminate narrow an...,True,True
3,NaN,False,NaN
4,NaN,False,NaN
5,Bundle nearby buildings into larger blocks and...,False,NaN
6,NaN,True,True
7,NaN,False,NaN
8,NaN,False,NaN
9,NaN,False,NaN


In [20]:
# ===================== CELL 9 — Optional: merge sample metadata labels (collision-proof) =====================
import pandas as pd
from pathlib import Path

META_CSV = Path(CONFIG.PATHS.DATA_DIR) / "input" / "samples" / "metadata" / "meta.csv"
print("META_CSV:", META_CSV, "| exists:", META_CSV.is_file())

DO_MERGE_META = META_CSV.is_file()

if DO_MERGE_META:
    meta = pd.read_csv(META_CSV)

    required = {"sample_id", "operator", "intensity", "param_value"}
    missing = required - set(meta.columns)
    if missing:
        raise KeyError(f"meta.csv missing columns: {sorted(missing)}")

    if "tile_id" not in df.columns:
        print("df has no 'tile_id' column; skipping meta merge.")
    else:
        # normalize join keys
        df["tile_id_str"] = df["tile_id"].astype(str).str.strip().str.zfill(4)
        meta["sample_id_str"] = meta["sample_id"].astype(str).str.strip().str.zfill(4)

        meta_small = meta[["sample_id_str", "operator", "intensity", "param_value"]].copy()

        # Merge with suffixes to avoid overwriting any existing columns
        df = df.merge(
            meta_small,
            how="left",
            left_on="tile_id_str",
            right_on="sample_id_str",
            suffixes=("", "_meta"),
        ).drop(columns=["tile_id_str", "sample_id_str"], errors="ignore")

        # If df already had operator/intensity/param_value, they remain as-is,
        # and meta values land in operator_meta/intensity_meta/param_value_meta.
        # If df did NOT have them, then only the meta versions exist.
        for col in ["operator", "intensity", "param_value"]:
            meta_col = f"{col}_meta"
            if col not in df.columns and meta_col in df.columns:
                # no original -> promote meta to canonical name
                df[col] = df[meta_col]
            elif col in df.columns and meta_col in df.columns:
                # original exists -> fill missing originals from meta
                df[col] = df[col].where(df[col].notna(), df[meta_col])

        # optional: drop the *_meta columns (keep if you want audit)
        df = df.drop(columns=[c for c in ["operator_meta", "intensity_meta", "param_value_meta"] if c in df.columns])

        print("Meta merged.")
        if "operator" in df.columns:
            print("Matched operator rows:", int(df["operator"].notna().sum()), "/", len(df))
        else:
            print("'operator' column still missing after merge. Columns:", list(df.columns))

df.head(5)


META_CSV: /Users/amirdonyadide/Documents/GitHub/IMGOFUP/data/input/samples/metadata/meta.csv | exists: True
Meta merged.
Matched operator rows: 786 / 786


,timestamp,assignment_id,participant_id,tile_id,prompt_id,complete,remove,free_text,cleaned_text,param_value_x,...,step2_text,threshold_evidence,threshold_known_evidence,operator_y,intensity_y,param_value_y,operator,intensity,param_value,conflict_reason
0,2025-11-27 21:20:30.892,b9b3eea7-3fde-4320-8eb1-2f397fdb787d,0fa332ca-ab78-4e18-8d9c-74a21bdacc81,1304,1,True,False,الگویی رو نمیبینم اجتماع داریم ولی عمومی نیست ...,NaN,0.000,...,NaN,,,aggregate,medium,0.000,aggregate,medium,0.000,
1,2025-11-27 20:27:27.337,cb0ed21c-6777-48c3-b7be-425069f4f585,17b7bd76-8792-4806-b2a7-88e70cfb822d,433,2,True,True,remove most of the smaller and closely spaced ...,remove most of the smaller and closely spaced ...,0.094,...,remove most of the smaller and closely spaced ...,,,select,high,94.000,select,high,94.000,The prompt does not match the 'select' operato...
2,2025-12-06 20:11:54.205,87db583a-fca6-4858-b4bb-f25d05530b86,fc62e5f9-9578-4aeb-b194-2e52c2047769,1457,3,True,False,Remove small buildings and eliminate narrow an...,Remove small buildings and eliminate narrow an...,17.805,...,Remove small buildings and eliminate narrow an...,,,select,low,17.805,select,low,17.805,"The prompt suggests removing small buildings, ..."
3,2025-11-27 19:57:43.721,c82c8e11-48d8-4f74-af3f-d4aa95744c38,bc57129a-ef70-43b7-a511-4654c950d26c,1617,4,True,True,I want to use the map on the left side because...,NaN,0.000,...,NaN,,,aggregate,low,0.000,aggregate,low,0.000,
4,2025-11-27 19:59:48.805,2e6ba9a9-3beb-4bfa-979c-7d51eb097214,bc57129a-ef70-43b7-a511-4654c950d26c,1532,5,True,True,I want to use the map on the left side because...,NaN,6.309,...,NaN,,,displace,medium,6.309,displace,medium,6.309,


In [13]:
# ===================== CELL 10 — Optional: conflict detection (no threshold columns required) =====================
import pandas as pd

from imgofup.userstudy.prompt_cleaning import conflict_llm

DO_CONFLICT_CHECK = True  # set False to skip API calls

if DO_CONFLICT_CHECK:
    # We only require what we actually have now
    required_cols = ["cleaned_text", "operator", "intensity"]
    missing = [c for c in required_cols if c not in df.columns]
    if missing:
        raise KeyError(f"Missing required columns for conflict check: {missing}")

    # define "useful" rows
    if "complete" in df.columns:
        # robust bool-ish conversion
        useful_mask = df["complete"].astype(str).str.strip().str.lower().isin(["true", "1", "yes", "y", "t"])
    else:
        useful_mask = pd.Series([True] * len(df), index=df.index)

    # Only check rows that have the minimum required info
    todo = useful_mask & df["cleaned_text"].notna() & df["operator"].notna()

    # Create columns if missing (don’t wipe previous runs)
    if "conflict" not in df.columns:
        df["conflict"] = pd.NA
    if "conflict_reason" not in df.columns:
        df["conflict_reason"] = ""

    # Only run where conflict not computed yet
    todo = todo & df["conflict"].isna()

    print("Rows to check:", int(todo.sum()))

    if todo.any():
        idxs = df.index[todo].tolist()

        # ---- run LLM row-by-row with progress ----
        try:
            from tqdm.auto import tqdm
            iterator = tqdm(idxs, desc="Conflict check", total=len(idxs))
        except Exception:
            iterator = idxs

        conflicts = []
        reasons = []

        for i in iterator:
            row = df.loc[i]

            out = conflict_llm(
                prompt=row.get("cleaned_text"),
                operator=row.get("operator"),
                intensity=row.get("intensity"),
                param_value=row.get("param_value", ""),  # may not exist; okay
                threshold_exist=False,   # you removed threshold steps → treat as false
                threshold_known=False,   # you removed threshold steps → treat as false
                client=client,
                cfg=LLM,
            )

            conflicts.append(bool(out.get("conflict", True)))
            reasons.append(str(out.get("reason", "") or "").strip())

        df.loc[idxs, "conflict"] = conflicts
        df.loc[idxs, "conflict_reason"] = reasons

    print("\nConflict counts (checked rows only):")
    print(df.loc[df["conflict"].notna(), "conflict"].value_counts(dropna=False))


Rows to check: 613


Conflict check:   0%|          | 0/613 [00:00<?, ?it/s]


Conflict counts (checked rows only):
conflict
True     454
False    159
Name: count, dtype: int64


In [14]:
# ===================== CELL 11 — Save cleaned dataset (final artifact) =====================
from pathlib import Path

# Ensure output directory exists
OUT_CSV = Path(OUT_CSV)
OUT_XLSX = Path(OUT_XLSX)
OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
OUT_XLSX.parent.mkdir(parents=True, exist_ok=True)

# ---- Save CSV (always) ----
df.to_csv(OUT_CSV, index=False)

# ---- Save Excel (optional) ----
excel_ok = True
try:
    df.to_excel(OUT_XLSX, index=False)
except Exception as e:
    excel_ok = False
    print("Excel export failed (CSV is still saved):", e)

# ---- Report ----
print("\nCleaned user-study dataset saved")
print("Rows   :", len(df))
print("Columns:", len(df.columns))
print("CSV    :", OUT_CSV.resolve())
if excel_ok:
    print("XLSX   :", OUT_XLSX.resolve())

# ---- Quick sanity preview ----
display(df.tail(5))



Cleaned user-study dataset saved
Rows   : 786
Columns: 29
CSV    : /Users/amirdonyadide/Documents/GitHub/IMGOFUP/data/userstudy/UserStudy.cleaned.csv
XLSX   : /Users/amirdonyadide/Documents/GitHub/IMGOFUP/data/userstudy/UserStudy.cleaned.xlsx


,timestamp,assignment_id,participant_id,tile_id,prompt_id,complete,remove,free_text,cleaned_text,param_value_x,...,step2_text,threshold_evidence,threshold_known_evidence,operator_y,intensity_y,param_value_y,operator,intensity,param_value,conflict_reason
781,2025-12-09 20:15:34.224,cc14d19b-6c4e-47d9-9231-799229ff9399,1db7694b-b1f3-4686-adaf-5e9b445877eb,1525,782,True,False,Gaps between buildings that were closer than a...,NaN,7.000,...,NaN,,,aggregate,high,6.688,aggregate,high,6.688,
782,2025-12-09 20:21:24.828,40a633ca-5868-4fae-be43-adf6061bfd35,1db7694b-b1f3-4686-adaf-5e9b445877eb,1695,783,True,False,Nearby buildings have been merged.,NaN,7.031,...,NaN,,,aggregate,high,7.031,aggregate,high,7.031,
783,2025-11-27 22:07:19.464,be146861-a126-46da-aeb9-f1a777fc3d20,f0c1cad7-bca1-477d-ac2b-277eda631b5f,507,784,True,False,make very close parcels unified so they shape ...,make very close parcels unified so they shape ...,10.094,...,make very close parcels unified so they shape ...,,,aggregate,medium,10.094,aggregate,medium,10.094,The prompt does not match the operator label '...
784,2025-11-30 17:32:18.732,f0a689d3-3616-47a8-beed-8462801724c4,cb7887ec-b4f3-4727-a7f4-fee2a49dee20,515,785,True,True,elimination of small blocks used,NaN,753.846,...,NaN,,,select,high,753.846,select,high,753.846,
785,2025-12-05 21:40:49.685,a849ccb8-144b-4fd5-b323-f7f61277e210,de7d2edd-a061-43cf-ba8e-8d67e9ffe396,1595,786,True,False,increase the distance between the units,increase the distance between the units,10.353,...,increase the distance between the units,,,displace,high,10.353,displace,high,10.353,The prompt does not match the operator label '...
